# Testing my methodology

In [1]:
import os
import argparse
# set the directory to the root
# os.chdir("/Users/jeandurand/Documents/Masters Thesis/causal_bayes_opt")
os.chdir("/vol/bitbucket/jd123/causal_bayes_opt")

import numpy as np
import pandas as pd

from scripts.base_script import parse_args
from diffcbed.models.dag_bootstrap import DagBootstrap
from diffcbed.strategies import PolicyOptNMC
from diffcbed.envs.graph_to_env import GraphStructureEnv
from graphs.toy_graph import ToyGraph
from graphs.graph_6_nodes import Graph6Nodes
from graphs.graph_5_nodes import Graph5Nodes
from graphs.graph_erdos_renyi import ErdosRenyiGraph
from graphs.graph_chain import ChainGraph
from algorithms.PARENT_algorithm import PARENT
from graphs.data_setup import setup_observational_interventional

No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.


cuda is available True


In [2]:
args = parse_args()
# graph = ErdosRenyiGraph(num_nodes=20)
# graph = ChainGraph(num_nodes=20)
# graph = Graph6Nodes()
# graph = ToyGraph()
graph = Graph5Nodes()
args.num_nodes = len(graph.variables)
graph_env = GraphStructureEnv(graph, args)
D_O, D_I, _ = setup_observational_interventional(graph_type=None, graph=graph, n_obs=1000, n_int=0)
model = DagBootstrap(graph_env, args)
strategy = PolicyOptNMC(model, graph_env, args)

INFO:root:Initializing the Toy Graph Structures
INFO:root:Sampling the observational data
INFO:root:Sampling the interventional data
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


Expected degree: 0.6666666666666666


2024-07-09 08:14:00.612557: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [3]:
my_algo = PARENT(graph, graph_env, model, strategy)
my_algo.set_values(D_O, D_I)
my_algo.run_algorithm()

0.10649121661823448 0.9151926239061877 0.05
0.6205840314428468 0.5348733489992731 0.05
[False False]


# Testing out the DAG bootstrap posterior model

In [3]:
import os
# set the directory to the root
# os.chdir("/Users/jeandurand/Documents/Masters Thesis/causal_bayes_opt")
os.chdir("/vol/bitbucket/jd123/causal_bayes_opt")
from diffcbed.strategies import PolicyOptNMC
from diffcbed.models import DagBootstrap
from algorithms.BOED_algorithm import BOED
from algorithms.PARENT_algorithm import PARENT
from graphs.toy_graph import ToyGraph
from diffcbed.envs.graph_to_env import GraphStructureEnv
from scripts.base_script import parse_args

In [4]:
args = parse_args()
graph = ToyGraph()
args.num_nodes = len(graph.variables)
graph_env = GraphStructureEnv(graph, args)
graph_variables = graph_env.nodes
node_ranges = []
interventional_ranges = graph.get_interventional_range()
for node in graph.variables:
    if node in interventional_ranges.keys():
        node_ranges.append(tuple(interventional_ranges[node]))
    else:
        node_ranges.append((0, 0))
args.node_range = node_ranges
graph_env = GraphStructureEnv(graph, args)

strategy_name = "policyoptnmc"
model = DagBootstrap(graph_env, args)
strategy = PolicyOptNMC(model, graph_env, args)
parent = PARENT(graph, graph_env, model)

INFO:root:Initializing the Toy Graph Structures


Expected degree: 0.6666666666666666
Expected degree: 0.6666666666666666


RuntimeError: Unable to initialize backend 'cuda': FAILED_PRECONDITION: No visible GPU devices. (you may need to uninstall the failing plugin package, or set JAX_PLATFORMS=cpu to skip this backend.)

In [8]:
%tb

SystemExit: 2